In [17]:
import psycopg2
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
import string
import numpy as np
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [18]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

In [19]:
df = retrieve_reviews('Samsung Electronics UN65MU6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')

In [20]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,475,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,1,5.0 out of 5 stars,"August 28, 2017",James Barrow,Someone here complained that the television is...,"Before I purchased this television, I read alm...",Verified Purchase Helpful No...
1,476,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,2,5.0 out of 5 stars,"June 18, 2017",Raul G.,"Incredible Picture Quality, Excellent Features...",Its been about seven years since I purchased m...,TOP 100 REVIEWERVINE VOICESize: 50-Inch|Style: TV


In [21]:
columns = ['id','p_name','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']
df.columns = columns
df.head(2)

,id,p_name,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,475,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,1,5.0 out of 5 stars,"August 28, 2017",James Barrow,Someone here complained that the television is...,"Before I purchased this television, I read alm...",Verified Purchase Helpful No...
1,476,Samsung Electronics UN65MU6300 65-Inch 4K Ultr...,https://www.amazon.com/Samsung-Electronics-UN6...,1,2,5.0 out of 5 stars,"June 18, 2017",Raul G.,"Incredible Picture Quality, Excellent Features...",Its been about seven years since I purchased m...,TOP 100 REVIEWERVINE VOICESize: 50-Inch|Style: TV


In [22]:
df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new = df_new[['rating','r_name','r_text']]
df_new.head(3)

,rating,r_name,r_text
0,1,James Barrow,"Before I purchased this television, I read alm..."
1,1,Raul G.,Its been about seven years since I purchased m...
2,1,Y. L.,WATCH OUT FOR the TV ADS AGREEMENT DURING SET ...


In [23]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three


In [25]:
df_text = df_new[['r_text']]

In [26]:
df_text.info()

data = df_text.applymap(cleaning)['r_text']
text = data
text_list = [i.split() for i in text]
len(text_list)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1259 entries, 0 to 1666
Data columns (total 1 columns):
r_text    1259 non-null object
dtypes: object(1)
memory usage: 19.7+ KB


1259

In [27]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
 
# Let's see how the first document in the corpus looks like in different topic spaces
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])
 

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


(1259, 10)
(1259, 10)
(1259, 10)
[0.00080649 0.3571126  0.00080657 0.41996758 0.00080647 0.00080654
 0.00080646 0.0677012  0.00080663 0.15037947]
[0.30182594 0.         0.04809768 0.0751346  0.         0.
 0.         2.82257331 0.         0.41976657]
[ 6.92725245 -0.58706842  0.81203349  7.64948384  5.1542544   4.21055833
 -2.7105833  -3.44257502  2.74524862 -3.15920425]


In [28]:
type(data_vectorized.todense().shape)

tuple

In [29]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('awesome', 58.308884141054605), ('thank', 17.354887448300357), ('soundbar', 8.331284357200339), ('beat', 7.386884958541158), ('satisfied', 6.100348719922546), ('samsung', 5.459753035756146), ('watching', 3.9341620307530967), ('electronics', 3.546766779474708), ('xbox', 3.429365169560511), ('place', 3.356033797157945)]
Topic 1:
[('samsung', 284.7890012205421), ('screen', 107.89792522753632), ('day', 65.34729702721044), ('time', 61.105051942761826), ('service', 57.890777507846146), ('box', 57.366470058319116), ('work', 55.67282466529284), ('problem', 55.42818671386725), ('issue', 51.67935402744817), ('support', 50.058488378634664)]
Topic 2:
[('natural', 5.858533674094935), ('adjust', 4.574920095273889), ('setting', 3.230975570809726), ('liked', 3.2122588449587925), ('learn', 2.884746819440487), ('brightness', 2.106683013237634), ('disappointed', 1.63592452356142), ('hour', 1.5548152432642766), ('standard', 1.1420664660301532), ('told', 1.1340493541201697)]
Topic 3:


In [30]:
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [31]:

svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [32]:

svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [33]:

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The economy is working better than ever"
x = lda_model.transform(vectorizer.transform([text]))[0]
print(x, x.sum())

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


[0.03333951 0.03333333 0.336655   0.03333498 0.03333333 0.03334505
 0.39665405 0.03333807 0.03333333 0.03333333] 1.0


In [34]:
import ipywidgets
ipywidgets.__version__
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
7      30.818288        1       1  110.219917  -90.441475
5      22.808126        1       2   12.363756  -40.379673
6      16.061355        1       3  -87.682426   98.413261
2      14.176947        1       4  -82.913422  -95.197456
3       9.876247        1       5  110.998131    6.443006
0       2.281279        1       6    7.726853  147.759979
1       2.083740        1       7  -88.453766    1.530137
4       0.855968        1       8  105.450851  103.171684
8       0.531680        1       9   10.178093   48.352802
9       0.506370        1      10   14.809695 -139.786987, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
323   Default  492.000000       great  492.000000  30.0000  30.0000
644   Default  473.000000     samsung  473.000000  29.0000  29.0000
650   Default   58.000000         say   58.000000  28.0000  28.0000
536   Default  529.000000     picture  529.000000  27.0000  27.0000
435   Default  202.000000        love  202.000000  26.0000  26.0000
529   Default   53.000000     perfect   53.000000  25.0000  25.0000
579   Default   71.000000    purchase   71.000000  24.0000  24.0000
238   Default  183.000000        easy  183.000000  23.0000  23.0000
35    Default   89.000000        apps   89.000000  22.0000  22.0000
861   Default  181.000000        work  181.000000  21.0000  21.0000
222   Default   39.000000     display   39.000000  20.0000  20.0000
833   Default   34.000000        wall   34.000000  19.0000  19.0000
67    Default   84.000000        best   84.000000  18.0000  18.0000
528   Default   45.000000      people   45.000000  17.0000  17.0000
605   Default   66.000000   recommend   66.000000  16.0000  16.0000
252   Default   93.000000   excellent   93.000000  15.0000  15.0000
699   Default  177.000000       smart  177.000000  14.0000  14.0000
470   Default   29.000000       mount   29.000000  13.0000  13.0000
758   Default   91.000000  television   91.000000  12.0000  12.0000
671   Default  116.000000     setting  116.000000  11.0000  11.0000
334   Default   83.000000       happy   83.000000  10.0000  10.0000
669   Default   75.000000     service   75.000000   9.0000   9.0000
339   Default   77.000000        hdmi   77.000000   8.0000   8.0000
569   Default   98.000000     product   98.000000   7.0000   7.0000
189   Default  103.000000         day  103.000000   6.0000   6.0000
633   Default   67.000000      review   67.000000   5.0000   5.0000
463   Default   77.000000       model   77.000000   4.0000   4.0000
51    Default   79.000000     awesome   79.000000   3.0000   3.0000
639   Default   54.000000        room   54.000000   2.0000   2.0000
486   Default   93.000000        nice   93.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
661   Topic10    0.120398        self    6.188712   1.3460  -6.7779
613   Topic10    0.122849      remote  281.756443  -2.4522  -6.7578
861   Topic10    0.122269        work  181.171657  -2.0153  -6.7625
82    Topic10    0.121883         box  130.368051  -1.6894  -6.7657
638   Topic10    0.121789        roku   20.222120   0.1734  -6.7665
633   Topic10    0.121716      review   67.448449  -1.0318  -6.7670
307   Topic10    0.121505      gaming   31.854652  -0.2833  -6.7688
162   Topic10    0.121412        cord   18.105578   0.2809  -6.7696
598   Topic10    0.121335      really  115.067217  -1.5691  -6.7702
101   Topic10    0.121333       cable   97.907763  -1.4076  -6.7702
65    Topic10    0.121278     bedroom   21.475158   0.1091  -6.7707
480   Topic10    0.121090      needed   17.711347   0.3002  -6.7722
782   Topic10    0.121063     trouble   11.197894   0.7585  -6.7724
467   Topic10    0.121063     monitor   44.355309  -0.6180  -6.7724
769   Topic10    0.121055       thing   91.606519  -1.3434  -6.77